<a href="https://colab.research.google.com/github/mejian1/ExopherGeneExpressionProfiling/blob/main/fastingGene_Clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

try:
    # 1. Load and Prepare Data (same as before)
    new_column_names = [
        'WBID', 'Gene_ID', 'Public_Name',
        'Fed_avg', 'Fed_sd', 'Fed_CV',
        '3h_fast_avg', '3h_fast_sd', '3h_fast_CV',
        '4h_fast_avg', '4h_fast_sd', '4h_fast_CV',
        '6h_fast_avg', '6h_fast_sd', '6h_fast_CV'
    ]
    df = pd.read_csv('harvardfastingDGEnotToptable.csv', header=None, skiprows=2, names=new_column_names)

    # Log2 Transform Average Expression
    avg_cols = ['Fed_avg', '3h_fast_avg', '4h_fast_avg', '6h_fast_avg']
    df_avg_log2 = np.log2(df[avg_cols] + 1)

    # Standardize the data
    scaler = StandardScaler()
    df_scaled_values = scaler.fit_transform(df_avg_log2.T).T
    df_scaled = pd.DataFrame(df_scaled_values, index=df_avg_log2.index, columns=df_avg_log2.columns)

    # 2. Perform K-Means Clustering (with k=6)
    k_optimal = 6
    kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(df_scaled)

    # 3. Create and Export the Gene Cluster File
    # Create a new DataFrame with the gene identifiers and their assigned cluster.
    df_clusters = df[['WBID', 'Public_Name']].copy()
    df_clusters['cluster'] = cluster_labels

    # Save the DataFrame to a CSV file.
    df_clusters.to_csv('gene_clusters.csv', index=False)

    print("Successfully exported the gene clusters to 'gene_clusters.csv'.")
    # Also, print the head of the file to show the user.
    print("\nHead of the exported file (gene_clusters.csv):")
    print(df_clusters.head())

    # Print the size of each cluster
    print("\nNumber of genes in each cluster:")
    print(df_clusters['cluster'].value_counts().sort_index())


except FileNotFoundError:
    print("The file 'harvardfastingDGEnotToptable.csv' was not found. Please make sure the file is in the correct directory.")
except Exception as e:
    print(f"An error occurred: {e}")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


```text?code_stderr&code_event_index=2
Traceback (most recent call last):
  File "<string>", line 46
    df = pd.read_csv('harvardfastingDGEnotToptable.csv', header=1)
IndentationError: expected an indented block after 'if' statement on line 40

```

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

try:
    # 1. Load and Prepare Data
    new_column_names = [
        'WBID', 'Gene_ID', 'Public_Name',
        'Fed_avg', 'Fed_sd', 'Fed_CV',
        '3h_fast_avg', '3h_fast_sd', '3h_fast_CV',
        '4h_fast_avg', '4h_fast_sd', '4h_fast_CV',
        '6h_fast_avg', '6h_fast_sd', '6h_fast_CV'
    ]
    df = pd.read_csv('harvardfastingDGEnotToptable.csv', header=None, skiprows=2, names=new_column_names)

    # Log2 Transform Average Expression
    avg_cols = ['Fed_avg', '3h_fast_avg', '4h_fast_avg', '6h_fast_avg']
    df_avg = df[avg_cols]
    df_avg_log2 = np.log2(df_avg + 1)

    # Standardize the data
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df_avg_log2.T).T
    df_scaled = pd.DataFrame(df_scaled, index=df_avg_log2.index, columns=df_avg_log2.columns)

    # 2. Perform K-Means Clustering (using k=6 as determined previously)
    k_optimal = 6
    kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(df_scaled)

    # 3. Create and Export Cluster Assignment File
    df_clusters = df[['WBID', 'Public_Name']].copy()
    df_clusters['cluster'] = cluster_labels

    # Save to CSV
    df_clusters.to_csv('gene_clusters.csv', index=False)

    print("Gene cluster assignments have been successfully exported to 'gene_clusters.csv'")
    print(df_clusters.head())
    print("\nCluster sizes:")
    print(df_clusters['cluster'].value_counts())


except FileNotFoundError:
    print("The file 'harvardfastingDGEnotToptable.csv' was not found. Please make sure the file is in the correct directory.")
except Exception as e:
    print(f"An error occurred: {e}")

```text?code_stdout&code_event_index=6
Gene cluster assignments have been successfully exported to 'gene_clusters.csv'
             WBID Public_Name  cluster
0  WBGene00011491     T05F1.5        0
1  WBGene00015353    C02F5.13        1
2  WBGene00044067    zipt-7.1        3
3  WBGene00020283     T06A4.3        1
4  WBGene00011376       gla-3        0

Cluster sizes:
4    3171
0    2662
5    1494
3    1311
1    1140
2     840
Name: cluster, dtype: int64

```

[file-tag: code-generated-file-0-1755904171505790838]

Of course. I've exported the gene cluster assignments to a CSV file for you. You can download it here:

**[gene\_clusters.csv](https://www.google.com/search?q=download:generated:gene_clusters.csv)**

This file contains the WormBase ID (WBID), the public gene name, and the cluster number (0 through 5) that I assigned to each gene.

-----

### Functional Enrichment Analysis: The "So What?" 生物

Now we get to the most interpretive and biologically meaningful part of the analysis: figuring out what these groups of co-regulated genes actually *do*. This is where we connect the dots from raw expression changes to biological function. Since you're a fellow geneticist, you'll appreciate how this step helps us build testable hypotheses.

#### 1\. Gene Ontology (GO) Analysis

The first thing we'll do is perform a **Gene Ontology (GO) analysis** on each cluster. GO provides a controlled vocabulary to describe the attributes of genes and gene products in any organism. It's broken down into three domains:

  * **Biological Process (BP):** The larger biological programs that a gene product is involved in, like "aging" or "autophagy."
  * **Molecular Function (MF):** The specific molecular activities of a gene product, such as "protein kinase activity" or "DNA binding."
  * **Cellular Component (CC):** The locations in the cell where a gene product is active, like "nucleus" or "mitochondrion."

For each of our gene clusters, we'll ask: "Are there any GO terms that are statistically overrepresented in this list of genes compared to all the genes in the *C. elegans* genome?"

For example, we might find that the genes in **Cluster 2** (the ones that are strongly and consistently upregulated during fasting) are significantly enriched for GO terms like "autophagy," "lipid metabolism," and "response to starvation." This would provide strong evidence that these are the key biological processes being activated to cope with the lack of nutrients.

#### 2\. Pathway Analysis (KEGG)

Next, we can perform a **pathway analysis**, often using the **Kyoto Encyclopedia of Genes and Genomes (KEGG)** database. While GO analysis tells us about broad biological themes, KEGG analysis is more specific and tells us if our genes are enriched in well-defined metabolic or signaling pathways.

For instance, we might discover that the downregulated genes in **Cluster 0** are significantly enriched in pathways related to "ribosome biogenesis" or "cell cycle," which makes perfect biological sense—when starved, an animal will shut down costly processes like growth and reproduction to conserve energy.

#### Recommended Tools for *C. elegans*

Given your background in bioinformatics using Python and R, you have several excellent options for performing this analysis. You can simply take the lists of WBIDs from the CSV file for each cluster and plug them into these tools:

  * **Web-Based (Easy and Quick):**

      * **WormBase Gene Set Enrichment Analysis:** The official resource for *C. elegans*. It has a built-in GO Term Finder that is specifically tailored for worm data. ([https://wormbase.org/tools/enrichment/tea/tea.cgi](https://wormbase.org/tools/enrichment/tea/tea.cgi))
      * **DAVID:** A very popular and comprehensive tool that provides enrichment analysis for GO, KEGG, and many other functional categories. ([https://david.ncifcrf.gov/](https://david.ncifcrf.gov/))
      * **g:Profiler:** A powerful and user-friendly web server for functional enrichment analysis. ([https://biit.cs.ut.ee/gprofiler/gost](https://biit.cs.ut.ee/gprofiler/gost))

  * **Programmatic (Reproducible and Customizable):**

      * **R:** The `clusterProfiler` package is the gold standard for this type of analysis in R. It's incredibly powerful, flexible, and produces great visualizations.
      * **Python:** The `gseapy` library is a great Python wrapper for performing gene set enrichment analysis.

By performing these analyses, you'll be able to move from a list of thousands of changing genes to a clear, actionable story about the molecular and cellular response of *C. elegans* to fasting.